In [ ]:
# ======================================
# COMPILADOR DE REGISTROS T6 (.gpkg → .xlsx → .gpkg)
# ======================================

import os
import fiona
import geopandas as gpd
import pandas as pd
import numpy as np

# Ruta a la carpeta con los GPKG (¡MODIFICA SOLO ESTA LÍNEA!)
ruta_archivos = r"C:\Users\carlo\Downloads\Registros\02_05_2025_Diani"

# Nombres de salida
nombre_excel = "02_05_2025_Diani.xlsx"
nombre_gpkg = "02_05_2025_Diani.gpkg"

# Orden esperado de columnas
orden_columnas = [
    "Clave de excavación", "Tramo", "ID Monumento", "No. Arqueólogo", "Punto de georreferencia",
    "X", "Y", "Z", "Código", "ID topógrafo", "No. Bolsa", "Procedencia",
    "Contexto de excavación", "Tipo de excavación", "Coordenada Y", "Coordenada X",
    "Capa", "Material", "Materia prima", "Asociación",
    "Descripción general del punto", "Descripción del estrato", "Fecha de registro",
    "FOTO_1", "FOTO_2", "FOTO_3", "FOTO_4", "Control de cambios", "ID Enlace"
]

# Función para normalizar nombres de columnas
def normalizar_nombre(nombre):
    return " ".join(
        nombre.strip().lower()
        .replace("á", "a").replace("é", "e").replace("í", "i")
        .replace("ó", "o").replace("ú", "u").replace("_", " ").replace(".", "").split()
    )

# Diccionario de alias de columnas
alias_columnas = {
    "calve de excavacion": "Clave de excavación",
    "clave de excavacion": "Clave de excavación",
    "clave excavacion": "Clave de excavación",
    "id mon": "ID Monumento",
    "tramo": "Tramo",
    "x": "X", "y": "Y", "z": "Z",
    "codigo": "Código",
    "id arqlg": "No. Arqueólogo",
    "id topo": "ID topógrafo", "id topografo": "ID topógrafo",
    "procedencia": "Procedencia",
    "contexto excavacion": "Contexto de excavación",
    "punto georreferencia": "Punto de georreferencia",
    "no. bolsa": "No. Bolsa", "no bolsa": "No. Bolsa", "numero de bolsa": "No. Bolsa",
    "numero_bolsa": "No. Bolsa", "no. bolsa ": "No. Bolsa",
    "desc punto": "Descripción general del punto", "descripcion general del punto": "Descripción general del punto",
    "desc estrato": "Descripción del estrato", "descripcion del estrato": "Descripción del estrato",
    "id enlace": "ID Enlace", "enlace": "ID Enlace",
    "control cambios": "Control de cambios",
    "asociacion": "Asociación",
    "tipo de excavacion": "Tipo de excavación", "tipo excavacion": "Tipo de excavación",
    "coord y": "Coordenada Y", "coord x": "Coordenada X",
    "capa": "Capa", "material": "Material", "materia prima": "Materia prima",
    "fecha registro": "Fecha de registro",
    "foto 1": "FOTO_1", "foto 2": "FOTO_2", "foto 3": "FOTO_3", "foto 4": "FOTO_4",
}

# ======================================
# CARGA Y NORMALIZACIÓN DE LOS ARCHIVOS
# ======================================

registros = []

for archivo in os.listdir(ruta_archivos):
    if archivo.endswith(".gpkg"):
        path = os.path.join(ruta_archivos, archivo)
        try:
            capas = fiona.listlayers(path)
            for capa in capas:
                gdf = gpd.read_file(path, layer=capa)
                columnas_norm = {
                    col: alias_columnas.get(normalizar_nombre(col), col)
                    for col in gdf.columns
                }
                gdf = gdf.rename(columns=columnas_norm)

                for col in orden_columnas:
                    if col not in gdf.columns:
                        gdf[col] = None

                gdf = gdf[orden_columnas]
                gdf["__archivo"] = archivo
                registros.append(gdf)
        except Exception as e:
            print(f"⚠️ Error en {path}: {e}")

if not registros:
    print("⚠️ No se generó ningún compilado.")
    exit()

# ======================================
# UNIR Y PROCESAR LOS DATOS
# ======================================

df = pd.concat(registros, ignore_index=True)

# Convertir X, Y a numérico
df["X"] = pd.to_numeric(df["X"], errors="coerce")
df["Y"] = pd.to_numeric(df["Y"], errors="coerce")

#Forzar a numerico
df["Coordenada X"] = pd.to_numeric(df["Coordenada X"], errors="coerce")
df["Coordenada Y"] = pd.to_numeric(df["Coordenada Y"], errors="coerce")

# Validar inversión de "Coordenada X" y "Coordenada Y"
cond_x = (df["Coordenada X"] < 200000) | (df["Coordenada X"] > 800000)
cond_y = (df["Coordenada Y"] < 1500000) | (df["Coordenada Y"] > 2400000)
invertir = cond_x & cond_y

if invertir.any():
    df.loc[invertir, ["Coordenada X", "Coordenada Y"]] = df.loc[invertir, ["Coordenada Y", "Coordenada X"]].values
    print(f"🔄 Coordenadas invertidas en {invertir.sum()} registros")

# Corrección de pares (solo para Coordenada X y Coordenada Y)
def corregir_par(valor):
    if pd.isna(valor):
        return np.nan
    valor = int(abs(valor))
    return valor if valor % 2 == 0 else valor - 1

df["Coordenada X"] = df["X"].apply(corregir_par)
df["Coordenada Y"] = df["Y"].apply(corregir_par)

# Guardar el Excel
df.to_excel(nombre_excel, index=False)
print(f"✅ Excel generado: {nombre_excel}")

# ======================================
# CONVERTIR A GeoDataFrame Y GUARDAR GPKG
# ======================================

# Formatear campos especiales
df["ID Monumento"] = df["ID Monumento"].astype(str).str.zfill(5)
df["Punto de georreferencia"] = df["Punto de georreferencia"].astype(str).str.zfill(7)
df["Fecha de registro"] = pd.to_datetime(df["Fecha de registro"], errors='coerce').dt.date

# Asegurar Z numérico
df["Z"] = pd.to_numeric(df["Z"], errors="coerce").fillna(0)

# Crear geometría 3D
gdf_salida = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["X"], df["Y"], df["Z"]),
    crs="EPSG:6371"  # ITRF2008 / UTM zone 16N
)

# Guardar GPKG
gdf_salida.to_file(nombre_gpkg, driver="GPKG", index=False)
print(f"✅ GeoPackage generado: {nombre_gpkg}")
